# Convolutional Neural Networks
---
In this notebook, we will explore how to use a **Convolutional Neural Network (CNN)** to classify images from the **CIFAR-10 dataset**.

The CIFAR-10 dataset consists of **60,000 32x32 color images**, divided into **10 classes** such as airplanes, cars, birds, cats, and more. CNNs are particularly effective for this task as they are designed to process grid-like data, such as images, and can efficiently capture spatial hierarchies in the data.

We will:
1. Build a CNN architecture tailored for CIFAR-10.
2. Train the network to classify images into their respective categories.
3. Evaluate the model’s performance to assess its accuracy.
4. Use a tuner for hyperparameter optimization



## 1. Python packages

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np

## 2. Getting dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
plt.figure(figsize=(1,1))
plt.imshow(X_train[100])
plt.show()

## 3. Data preprocessing

In [ ]:
num_classes = 10
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

input_shape = X_train.shape[1:]
X_train = X_train/255
X_test = X_test/255


In [ ]:
X_train.shape

## 4. Building the CNN

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu",
                      kernel_initializer=tf.keras.initializers.random_normal),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),

        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


In [ ]:
model.summary()

In [ ]:
batch_size = 128
epochs = 25

model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(), metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

## 5. Evaluating the CNN's performance

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
prediction = model.predict(X_test)
print("La predicción para cada imagen es un vector con 10 componentes")
print(prediction[0])

In [ ]:
labels = {0: "T-shirt/top", 1: "trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 5: "Sandal", 6: "Shirt",
         7: "Sneaker", 8:"Bag", 9: "Ankle boot"}

In [ ]:
from sklearn.metrics import classification_report
d = classification_report(y_test.argmax(axis=1), prediction.argmax(axis=1),  target_names=labels.values())

print(d)

In [ ]:
for i in range(1, 7):
    index_max = np.argmax(prediction[i-1,:])
    print("image ", i, "predicts ", labels[index_max])
    plt.subplot(2, 3, i)
    plt.imshow(X_test[i-1,:,:,0], cmap="gray")

## 6. Using keras_turner for hyperparameter optimization

In [ ]:
from keras_tuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(hp.Int('input_units',
                                           min_value=64,
                                           max_value=216,
                                           step=32),
                                    (3,3),
                                     input_shape=input_shape))


    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size= (2, 2)))
    model.add(tf.keras.layers.Flatten())
    hp_dense= hp.Int('1l', min_value = 128, max_value = 512, step = 32)
    model.add(tf.keras.layers.Dense(units = hp_dense, activation='relu'))
    hp_dense2 = hp.Int ('12', min_value = 128, max_value = 512, step = 32)
    model.add(tf.keras.layers.Dense(units = hp_dense, activation='relu'))

    hp_lr = hp.Choice('learning_rate', values= [0.001, 0.0001])

    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer="adam",
                  loss = "categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
import keras_tuner

In [ ]:
tuner = RandomSearch(
    build_model,
    objective=keras_tuner.Objective('val_accuracy', direction='max'),
    max_trials = 5,
    executions_per_trial = 2,
    directory = "",
    project_name = "")

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

In [ ]:
y_test.shape

In [ ]:
tuner.search_space_summary()

In [ ]:
final_model = tuner.get_best_models(num_models=1)

In [ ]:
best_model = final_model[0]

In [ ]:
best_model.summary()

In [ ]:
score2 = best_model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])